In [ ]:

from deephive.environment.utils import parse_config
from datetime import datetime
import os 
import numpy as np
# from deephive.environment.deephive_utils import *
import torch
from deephive.environment.optimization_environment import OptimizationEnv
from deephive.policies.mappo import MAPPO
from deephive.train_test_utils import train, test, initialize, initialize_logger, get_action, get_direct_action, mean_confidence_interval
# Set print options
np.set_printoptions(suppress=True, precision=4)
from dotenv import load_dotenv
api_token = os.environ.get("NEPTUNE_API_TOKEN")
load_dotenv()

In [ ]:
config_path = "../config/exp_config.json"
model_path = "../models/pbest_unfreeze.pth"
model_path_2 = "../models/gbest.pth"
config = parse_config(config_path)
config['use_gbest'] = True
config["use_optimal_value"] = True
config["log_scale"] = False 
config["include_gbest"] = False
config["negative"] = True
if config["include_gbest"]:
    config["obs_dim"] = 11
else:
    config["obs_dim"] = 9
config["ep_length"] = 25

config["min_action_std"] = 0.001
config["action_std"] = 0.2
config["variable_std"]  = False
config["update_timestep"] = 2
config["decay_rate"] = 0.99
config["log_interval"] = 500
config["decay_interval"] = 5
config["save_interval"] = 200
mode = "train"
config["test_decay_rate"] = 0.9
config["test_decay_start"] = 0

config["reward_scheme"] = "FullRewardScheme2"
config["n_episodes"] = 5000
config["plot_gif"] = True
config["plot_gbest"] = True
mode = 'test'

In [ ]:
MODEL_PATH = "/Users/elotech/Desktop/DeepHiveV2/notebooks/training_results/experiment_4/policy-0.pth"

In [ ]:
date = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
dimensions = [2]
dim_agents = {
    2: 50,
    5: 50,
    10: 100,
    20: 100,
    30: 40,
    50: 100
}

#function_ids = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
function_ids = ["f01", "f02", "f03", "f04", "f05", "f06", "f07", "f08", "f09", "f10"]
# function_opt_val = 100 * np.array(function_ids) + 100
config["test_ep_length"] = 100
config["n_agents"] = 10
config["n_dim"] = 2
config['objective_function'] = "CosineMixtureFunction" 
config["function_id"] = "f01"
env, agent_policy = initialize(config, mode=mode, model_path=[model_path, model_path_2])
if mode == "test":
    pbest_policy, gbest_policy = agent_policy[0], agent_policy[1]
else:
    agent_policy = agent_policy
obs, obs_std = env.reset()


In [ ]:

# for dim in dimensions:
#     config["n_agents"] = dim_agents[dim]
#     config["n_dim"] = dim
#     for function_id in function_ids[0:3]:
#         config["function_id"] = function_id
#         env, agent_policy = initialize(config, mode=mode, model_path=MODEL_PATH)
#         # if mode == "test":
#         #     pbest_policy, gbest_policy = agent_policy[0], agent_policy[1]
#         #     agent_policy = pbest_policy
#         # else:
#         #     agent_policy = agent_policy
#         agent_policy.load(MODEL_PATH)
#         try:
#             save_path = f"test_results/{date}/results{dim}D/function_{function_id}/"
#             os.makedirs(save_path, exist_ok=True)
#             optimal_positions = None
#             gbest_values = test(env, agent_policy, config, n_episodes=10, ep_length=config["test_ep_length"],
#                                 decay_rate=0.97, decay_start=20, debug=False, show_interval=10, 
#                                 fps=5, optimal_positions=optimal_positions, save_path=save_path, 
#                                 randomize=False)
#         except Exception as e:
#             print(e)
#             continue

In [ ]:
env, agent_policy = initialize(config, mode="train", model_path="../models/gbest.pth")
agent_policy.load(MODEL_PATH)
agent_policy.set_action_std(0.0001)

In [ ]:
def get_action(observation, agent_policy, env, observation_std=None, **kwargs):
    # Ensure observation_info is a numpy array
    
    if not isinstance(observation, np.ndarray):
        observation = np.array(observation)
        assert observation.shape[0] == env.n_dim, "Observation must have the same number of dimensions as the environment"

    # Initialize observation_std with zeros or use provided std, ensuring it matches the shape of observation
    if observation_std is None:
        observation_std = np.zeros_like(observation)
    else:
        observation_std = np.array(observation_std)

    # Flatten the observation and std arrays
    observation_flat = observation.reshape(env.n_dim * env.n_agents, -1)  # Flatten to 1D array
    observation_std_flat = observation_std.reshape(-1)  # Flatten to 1D array
    # Pass the entire flattened observation and std arrays to select_action
    action_flat = agent_policy.select_action(observation_flat, observation_std_flat)

    # Reshape the flattened action array back to the original (n_agents, n_dim) shape
    actions = action_flat.reshape(env.n_dim, env.n_agents).T  # Reshape to (n_agents, n_dim

    return actions  # Return the action


def get_direct_action(obs, obs_std, agent_policy):
    torch_obs = torch.FloatTensor(obs)
    torch_obs_std = torch.FloatTensor(obs_std)
    action = agent_policy.policy.act(torch_obs, torch_obs_std)
    return action

In [ ]:
policy = agent_policy
ep_length = 10
decay_start = 20
restart_interval = 5
current_action_std = 0.0001
policy.set_action_std(current_action_std)   
obs = env.reset()
obs, obs_std = obs
gbest_values = []
# for step in range(ep_length):
    
    # if step <= decay_start or step % restart_interval == 0:
    #     actions = get_action(obs, policy, env, obs_std, random=True) 
    

In [ ]:
env._get_actual_state()[:,-1]

In [ ]:
env.state[:,-1]

In [ ]:
actions = get_action(obs, policy, env, obs_std)

In [ ]:
obs, _, _, _ = env.step(actions)
obs = obs[0]

In [ ]:
actions = get_action(obs, policy, env, obs_std)
obs, _, _, _ = env.step(actions)
obs, obs_std = obs
gbest_values.append(env.gbest[-1])

In [ ]:
env._get_actual_state()[:,-1]

In [ ]:
env.state[:,-1]

In [ ]:
env.render()

In [ ]:
env._get_actual_state()[:,-1]

In [ ]:
env.state[:,-1]

In [ ]:
env.render()

In [ ]:
env.objective_function.tracker.count

In [ ]:
# Using only f5:
from cec2017.functions import f5
import numpy as np
samples = 3
dimension = 50
x = np.random.uniform(-100, 100, size=(samples, dimension))
val = f5(x)
for i in range(samples):
    print(f"f5(x_{i}) = {val[i]:.6f}")

# Using all functions:
from cec2017.functions import all_functions
for f in all_functions:
    x = np.random.uniform(-100, 100, size=(samples, dimension))
    val = f(x)
    for i in range(samples):
        print(f"{f.__name__}(x_{i}) = {val[i]:.6f}")

In [ ]:
_  = env.reset()

In [ ]:
env._get_actual_state()[:, -1]

In [ ]:
env.gbest

## DATA PROCESSING

In [ ]:
import pandas as pd

In [ ]:
base_data_path = "test_results/2024-02-29_15-29-49/"
ndim = 10
test_data_path = base_data_path + f"results{ndim}D"
df = pd.DataFrame(columns=["function_id", "mean", "lower", "upper", "optimum"])
for function_id in function_ids:
    try:
        data_path = f"{test_data_path}/function_{function_id}/gbest_values.npy"
        gbest_values = np.load(data_path) * -1
        mean_val, lower_val, upper_val = mean_confidence_interval(gbest_values)
        function_result = [f"function_{function_id}", mean_val[-1], lower_val[-1], upper_val[-1], function_opt_val[function_id]]
        df.loc[len(df)] = function_result
    except Exception as e:
        #print(e)
        continue

df.to_csv(f"{base_data_path}/results{ndim}D.csv", index=False)

In [ ]:
df

In [ ]:
min_p = -100
max_p = 100
x = np.linspace(min_p, max_p, 100)
y = np.linspace(min_p, max_p, 100)
X, Y = np.meshgrid(x, y)
Z = np.zeros_like(X)
for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        Z[i,j] = np.log10(-env.objective_function.evaluate(np.array([[X[i,j], Y[i,j]]])))
        print(Z[i,j])
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()

ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, Z, cmap='viridis')
plt.show()

In [ ]:
# plot a 2D surface plot showing the value of the objective function at different points in the search space
fig = plt.figure()
ax = fig.add_subplot(111)
ax.contourf(X, Y, Z, cmap='viridis')
# add a grid    
ax.grid(True)
# take 20 evenly spaced samples from the search space and write the value of the objective function at each point
# x = np.linspace(min_p, max_p, 100)
# y = np.linspace(min_p, max_p, 100)
# X, Y = np.meshgrid(x, y)
# Z = np.zeros_like(X)
# for i in range(X.shape[0]):
#     for j in range(X.shape[1]):
#         Z[i,j] = np.log10(-env.objective_function.evaluate(np.array([[X[i,j], Y[i,j]]])))
# ax.scatter(X, Y, c='r')
# for i in range(X.shape[0]):
#     for j in range(X.shape[1]):
#         ax.text(X[i,j], Y[i,j], f'{Z[i,j]:.2f}')
# show color bar
plt.colorbar(ax.contourf(X, Y, Z, cmap='viridis'))
plt.show()




## TRAINING

In [ ]:
mode = "train"

config['use_gbest'] = False
config["use_optimal_value"] = True
config["log_scale"] = True
config["include_gbest"] = True
if config["include_gbest"]:
    config["obs_dim"] = 5
else:
    config["obs_dim"] = 4
config["ep_length"] = 25

config["min_action_std"] = 0.00001
config["action_std"] = 0.3
config["variable_std"]  = False
config["update_timestep"] = 2
config["decay_rate"] = 0.99
config["log_interval"] = 500
config["decay_interval"] = 5
config["save_interval"] = 200
mode = "train"
config["test_decay_rate"] = 0.9
config["test_decay_start"] = 0

config["reward_scheme"] = "FullRewardScheme2"
config["n_episodes"] = 5000
config["plot_gif"] = True
config["plot_gbest"] = True


config["n_agents"] = 10
config["n_dim"] = 2
config['objective_function'] = "SphereFunction" 
config["function_id"] = 6
env, agent_policy = initialize(config, mode=mode, model_path=[model_path, model_path_2])
if mode == "test":
    pbest_policy, gbest_policy = agent_policy[0], agent_policy[1]
else:
    agent_policy = agent_policy
obs, obs_std = env.reset()

In [ ]:
neptune_logger = initialize_logger(api_token, "Deephive", config, mode=mode)
agent_policy = train(env, agent_policy, config, title="experiment_9", neptune_logger=neptune_logger)
print(agent_policy.action_std)
if neptune_logger is not None:
    neptune_logger.stop()

In [ ]:
agent_policy.load("training_results/experiment_9/policy-4800.pth")
# agent_policy.set_action_std(0.05)

In [ ]:
# obs, obs_std = env.reset()
# actions = get_action(obs, agent_policy, env, obs_std, random=False, pso_action=False, w=0.5, c1=1.5, c2=1.0)
# # print(actions)

# env.render()
# obs, _, _, _ = env.step(actions)
# obs, obs_std = obs
# env.render()
# actions = get_action(obs, agent_policy, env, obs_std, random=False, pso_action=False, w=0.5, c1=1.5, c2=1.0)
# print(actions)
# print(env._get_actual_state()[:, -1])
# print(env.state[:, -1])

In [ ]:
date = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
save_path = f"test_results/{date}/"
os.makedirs(save_path, exist_ok=True)
optimal_positions = None
env.n_agents = 100
gbest_values = test(env, agent_policy, config, n_episodes=20, ep_length=100,
                    decay_rate=0.95, decay_start=20, debug=False, show_interval=5, 
                    fps=5, optimal_positions=optimal_positions, save_path=save_path, 
                    randomize=False)

In [ ]:
env._get_actual_state()[:, -1]

In [ ]:
env.n_dim, env.n_agents

In [ ]:
print(env.n_agents)
env.n_agents = 100
_ = env.reset()
print(env.n_agents)

In [ ]:
avg, upper, lower = mean_confidence_interval(np.array(gbest_values))
print(avg[-1], upper[-1], lower[-1])


In [ ]:
index = np.random.choice(len(gbest_values))
print(f"Index: {index}")
gbest_values[index]

In [ ]:
env.reset()

In [ ]:
env.objective_function.tracker.count

## PSO VARIANTS

In [ ]:
config["test_ep_length"] = 50
config["ep_length"] = 100
config["n_agents"] = 100
config["n_dim"] = 2
config['objective_function'] = "CEC17" 
config["function_id"] = 0
config["log_scale"] = True
config["action_std"] = 0.02
env, agent_policy = initialize(config, mode=mode, model_path=[model_path, model_path_2])
if mode == "test":
    pbest_policy, gbest_policy = agent_policy[0], agent_policy[1]
else:
    agent_policy = agent_policy
obs, obs_std = env.reset()
MODEL_PATH = "training_results/experiment_9/policy-4800.pth"
agent_policy.load(MODEL_PATH)

In [ ]:
def get_action(observation, agent_policy, env, observation_std=None, random=False, env_random=False, pso_action=False, **kwargs):
    # Ensure observation_info is a numpy array
    
    if env_random:
        current_state =  env.state[:, :2]
        env._generate_init_state(count=True)
        new_state = env.state[:, :2]
        action = new_state - current_state
        return action
    
    if random:
        # Generate random actions
        action = np.random.uniform(-1, 1, size=(env.n_agents, env.n_dim))
        return action
        
    
    if not isinstance(observation, np.ndarray):
        observation = np.array(observation)
        assert observation.shape[0] == env.n_dim, "Observation must have the same number of dimensions as the environment"

    if pso_action:
        # grab the pso parameters from kwargs
        w = kwargs.get("w", 0.5)
        c1 = kwargs.get("c1", 1.5)
        c2 = kwargs.get("c2", 1.0)
        print(f"PSO action with w={w}, c1={c1}, c2={c2}")
        # grab the second, fourth and eight element of the observation for each dimension
        # that represents the previous velocity, best position and global best position
        # for each particle
        p_vel = observation[:, :,1]
        p_best = observation[:,:,3]
        g_best = observation[:,:,9]
        
        rp = np.random.uniform(-1, 1,size=env.n_dim)
        rg = np.random.uniform(-1, 1,size=env.n_dim)
        
        v = w * p_vel.T + c1 * rp * p_best.T + c2 * rg * g_best.T
        return v
    # Initialize observation_std with zeros or use provided std, ensuring it matches the shape of observation
    if observation_std is None:
        observation_std = np.zeros_like(observation)
    else:
        observation_std = np.array(observation_std)

    # Flatten the observation and std arrays
    observation_flat = observation.reshape(env.n_dim * env.n_agents, -1)  # Flatten to 1D array
    observation_std_flat = observation_std.reshape(-1)  # Flatten to 1D array
    # Pass the entire flattened observation and std arrays to select_action
    action_flat = agent_policy.select_action(observation_flat, observation_std_flat)

    # Reshape the flattened action array back to the original (n_agents, n_dim) shape
    actions = action_flat.reshape(env.n_dim, env.n_agents).T  # Reshape to (n_agents, n_dim

    return actions  # Return the action

In [ ]:
agent_policy.set_action_std(0.001)

In [ ]:

iters = 100
obs, obs_std = env.reset()
action_prob = 0.7
max_std = 0.2
min_std = 0.000001

for i in range(iters):
    actions = get_action(obs, agent_policy, env, obs_std)
    #pso_actions = get_action(obs, agent_policy, env, obs_std, random=True, pso_action=True, w=0.3, c1=0.5, c2=0.6)
    
    #This commented out code block is implementing a way to adjust the standard deviation of the actions taken by the agent based on a probability threshold.
    if np.random.random() > action_prob:
        agent_policy.set_action_std(max_std)
    else:
        agent_policy.set_action_std(min_std)
    
    obs, _, _, _ = env.step(actions)
    obs, obs_std = obs

print(env.gbest[-1])
    
# env.render()
env._get_actual_state()[:, -1]

In [ ]:
actions

In [ ]:
env.render(type="history", file_path="save_gif3.gif")